<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/NER/Custom%20NER%20Training%20using%20spaCy%20v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram K

### Imports

In [12]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import os

### Training Data

In [7]:
TRAIN_DATA = [
              ("Walmart is a leading e-commerce company", {"entities": [(0, 7, "ORG")]}),
              ("I reached Chennai yesterday.", {"entities": [(19, 28, "GPE")]}),
              ("I recently ordered a book from Amazon", {"entities": [(24,32, "ORG")]}),
              ("I was driving a BMW", {"entities": [(16,19, "PRODUCT")]}),
              ("I ordered this from ShopClues", {"entities": [(20,29, "ORG")]}),
              ("Fridge can be ordered in Amazon ", {"entities": [(0,6, "PRODUCT")]}),
              ("I bought a new Washer", {"entities": [(16,22, "PRODUCT")]}),
              ("I bought a old table", {"entities": [(16,21, "PRODUCT")]}),
              ("I bought a fancy dress", {"entities": [(18,23, "PRODUCT")]}),
              ("I rented a camera", {"entities": [(12,18, "PRODUCT")]}),
              ("I rented a tent for our trip", {"entities": [(12,16, "PRODUCT")]}),
              ("I rented a screwdriver from our neighbour", {"entities": [(12,22, "PRODUCT")]})
]

### Test Data

In [8]:
TEST_DATA = [
              ("I repaired my computer", {"entities": [(15,23, "PRODUCT")]}),
              ("I got my clock fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("I got my truck fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("Flipkart started it's journey from zero", {"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Max", {"entities": [(24,27, "ORG")]}),
              ("Flipkart is recognized as leader in market",{"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Swiggy", {"entities": [(24,29, "ORG")]})
]

### Initialize the blank en model

In [9]:
nlp = spacy.blank('en')

### Convert Data into spaCy format

In [21]:
class Convert:

    def __init__(self, nlp) -> None:
        self.nlp = nlp

    def to_spacy(self, data, split):
        
        db = DocBin()
        for text, annot in data:
            doc = self.nlp.make_doc(text)
            ents = []
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span is None:
                    print("Skipping", label)
                else:
                    ents.append(span)
            try:
                doc.ents = ents 
                db.add(doc)

            except Exception as e:
                print(f"Error:{e} {text}, {annot}")
        
        os.makedirs("./data", exist_ok=True)
        db.to_disk(f"data/{split}.spacy")

In [ ]:
convert = Convert(nlp)

convert.to_spacy(TRAIN_DATA, "train")
convert.to_spacy(TEST_DATA, "test")